## Responsible Data Use - Julia Lane Lecture Assignment

### Research Question: Which Community Districts in NYC show the highest number of complaints?

In [25]:
import os
import pandas as pd
import numpy as np
import re
PUIdata = os.getenv('PUIDATA')

In [26]:
os.system("curl -O https://data.cityofnewyork.us/resource/erm2-nwe9.csv")
os.system("mv erm2-nwe9.csv " + os.getenv("PUIDATA"))

0

In [169]:
data = pd.read_csv(os.getenv("PUIDATA") + '/' + 'erm2-nwe9.csv', usecols = ['Unique Key','Agency', 'Complaint Type',
                                'Community Board', 'Borough','Created Date', 'Closed Date', 'Resolution Description'], index_col=None)
data.head()

,Unique Key,Created Date,Closed Date,Agency,Complaint Type,Resolution Description,Community Board,Borough
0,34779271,11/15/2016 02:12:51 AM,NaN,NYPD,Noise - Vehicle,Your complaint has been received by the Police...,06 QUEENS,QUEENS
1,34775117,11/15/2016 02:11:40 AM,NaN,NYPD,Noise - Residential,Your complaint has been forwarded to the New Y...,03 BRONX,BRONX
2,34775418,11/15/2016 02:10:51 AM,NaN,DOHMH,Rodent,The Department of Health and Mental Hygiene wi...,06 BRONX,BRONX
3,34771971,11/15/2016 02:09:22 AM,11/15/2016 03:08:56 AM,NYPD,Noise - Vehicle,The Police Department responded and upon arriv...,03 QUEENS,QUEENS
4,34774246,11/15/2016 02:09:13 AM,NaN,DOHMH,Unsanitary Animal Pvt Property,The Department of Health and Mental Hygiene wi...,03 STATEN ISLAND,STATEN ISLAND


In [170]:
data.columns

Index([u'Unique Key', u'Created Date', u'Closed Date', u'Agency',
       u'Complaint Type', u'Resolution Description', u'Community Board',
       u'Borough'],
      dtype='object')

### Finding Measurement Errors

Looking to make sure the data is fully representative, checking for ommissions, duplications, content error, missing data, etc.

In [171]:
# Check if all Community Districts are represented in the Data 
data_2 = (data.groupby(['Community Board'], as_index=False).count())
data_2.head()

,Community Board,Unique Key,Created Date,Closed Date,Agency,Complaint Type,Resolution Description,Borough
0,0 Unspecified,30,30,11,30,30,26,30
1,01 BRONX,22,22,0,22,22,21,22
2,01 BROOKLYN,27,27,5,27,27,24,27
3,01 MANHATTAN,7,7,3,7,7,5,7
4,01 QUEENS,29,29,21,29,29,27,29


In [172]:
# How many unique values do we have
len(data_2)

64

#### We have 64 unique community boards in the data set

In [ ]:
# Why do we have so many? Some of them are unspecified, missing. Some might be invalid entries. 
# We should have 59 Community Districts.

In [173]:
# Check for duplicates? Are these plausible?
data_2_unique = data_2.drop_duplicates()
print (len(data_2))

64


#### The data doesn't show any duplicates. There are other issues with the data. For examples, some community boards are "Unspecified"

In [ ]:
# What about missing values? Can you detect any patterns? 

In [174]:
deleted = data_2.drop('Closed Date', 1)
null_values = deleted[pd.isnull(deleted).any(axis=1)]

In [175]:
data_2.isnull().sum() #sum up missing values

Community Board           0
Unique Key                0
Created Date              0
Closed Date               0
Agency                    0
Complaint Type            0
Resolution Description    0
Borough                   0
dtype: int64

In [176]:
data_2 = data_2[data_2['Community Board'].str.contains('Unspecified') !=True] #accouting for unspecificed values 

In [177]:
print(len(data_2))

60


In [178]:
data_2['Community Board'].drop_duplicates()

1             01 BRONX
2          01 BROOKLYN
3         01 MANHATTAN
4            01 QUEENS
5     01 STATEN ISLAND
6             02 BRONX
7          02 BROOKLYN
8         02 MANHATTAN
9            02 QUEENS
10    02 STATEN ISLAND
11            03 BRONX
12         03 BROOKLYN
13        03 MANHATTAN
14           03 QUEENS
15    03 STATEN ISLAND
16            04 BRONX
17         04 BROOKLYN
18        04 MANHATTAN
19           04 QUEENS
20            05 BRONX
21         05 BROOKLYN
22        05 MANHATTAN
23           05 QUEENS
24            06 BRONX
25         06 BROOKLYN
26        06 MANHATTAN
27           06 QUEENS
28            07 BRONX
29         07 BROOKLYN
30        07 MANHATTAN
31           07 QUEENS
32            08 BRONX
33         08 BROOKLYN
34        08 MANHATTAN
35           08 QUEENS
36            09 BRONX
37         09 BROOKLYN
38        09 MANHATTAN
39           09 QUEENS
40            10 BRONX
41         10 BROOKLYN
42        10 MANHATTAN
43           10 QUEENS
44         

In [179]:
print (len(data_2)) # there is one extra community board (60th value above) that is listed as 80 for Queens which doesn't repeat.

60


In [180]:
data_3 = data_2[data_2['Community Board'].str.contains('80') !=True] # dropping the random community board 80 

In [181]:
print(len(data_3)) #accounting for unspecified values gives us 59 

59


In [182]:
# Data Editing: Is it possible to replace missing values? Is it possible to use Complaint Type 
# to fill missings in Resolution Descriptor?

#### It is generally not a good practice to fill in missing values from one attribute of the dataset to another unless a clear relationship can be established, in which case, the missing values can be filled in based on the relationship.

In [183]:
# Generate marker for unplausible Community Districts
# How do these districts look like?

In [184]:
Qns = data_3['Community Board'].str.contains('8[0-9] QUEENS')
Bk = data_3['Community Board'].str.contains('5[0-9] BROOKLYN')
Bx = data_3['Community Board'].str.contains('2[0-9] BRONX')
Man = data_3['Community Board'] == '64 MANHATTAN'
Sta = data_3['Community Board'] == '95 STATEN ISLAND'
unspec = data_3['Community Board'].str.contains('Unspecified')

all_markers = (~Qns) & (~Bx) & (~Bk) & (~Man) & (~Sta) & (~unspec)

In [185]:
# Drop the marked districts
data_4 = data_3.loc[all_markers,]
data_4.shape

(59, 8)

In [186]:
# Produce your result: Generate an indicator which ranks the Community District by complaint numbers 
# on the Community district level

In [187]:
dist_level= data_4

In [188]:
dist_level_agg = dist_level[["Community Board","Complaint Type"]]
complaints = dist_level_agg.groupby('Community Board').count()
complaints.columns = ['Number of Complaints']

In [189]:
total_complaints = complaints.reset_index() #the number of complaints should be higher since its the aggregate of all the compplaints

In [190]:
# Safe reduce data 

total_complaints_by_commdist= total_complaints[['Community Board','Number of Complaints']].groupby('Community Board').count()
total_complaints_by_commdist.head().reset_index()

,Community Board,Number of Complaints
0,01 BRONX,1
1,01 BROOKLYN,1
2,01 MANHATTAN,1
3,01 QUEENS,1
4,01 STATEN ISLAND,1


In [191]:
total_complaints_by_commdist['Number of Complaints'].sum()

59

### Think about the interpretation of your results

What do you have to keep in mind when interpreting your results? Are they generable? Does the way the data is collected influence your results? To better inform city agancies it might be good to explore in more detail the underlying dempgraphics/infrastructure of a Community District becasue this might influence 311 calls. You can do this by merging external data on the Community District level to your analysis data

In [193]:
# Population by Community District
os.system("curl -O http://cosmo.nyu.edu/~fb55/PUI2016/data/Final_Demographics.csv" )
os.system("mv " + "Final_Demographics.csv " + os.getenv("PUIDATA"))
df_pop = pd.read_csv(PUIdata + "/Final_Demographics.csv")

In [194]:
df_pop.head()

,FIPS,cd_id,Total Population,Population Density (per sq. mile),% Total Population: Male,% Total Population: 18 to 24 Years,% Total Population: 25 to 34 Years,% Total Population: 35 to 44 Years,% Population 5 Years And Over: Speak Only English,% Population 5 Years And Over: Spanish or Spanish Creole,...,"Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations","% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations",% Employed Civilian Population 16 Years And Over: Service occupations,% Employed Civilian Population 16 Years And Over: Sales and office occupations,"% Employed Civilian Population 16 Years And Over: Farming, fishing, and forestry occupations","% Employed Civilian Population 16 Years And Over: Construction, extraction, and maintenance occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations"
0,3603701,BX08,106737,31229.95006,46.65,10.73,15.04,11.32,46.80,39.24,...,665,1518,42.66,28.95,20.89,0.24,2.65,4.60,1.40,3.20
1,3603702,BX12,134644,19966.67839,46.35,11.35,14.29,12.57,73.09,18.19,...,1156,4174,29.57,33.98,20.40,0.00,7.08,8.97,1.95,7.02
2,3603703,BX10,121209,12913.81703,45.20,8.62,13.74,12.78,61.79,26.43,...,941,3433,36.20,22.85,25.09,0.00,7.68,8.18,1.76,6.42
3,3603704,BX11,135839,35677.95453,50.09,8.10,17.43,14.09,43.22,36.45,...,2189,5592,30.06,27.86,22.24,0.00,7.03,12.81,3.60,9.20
4,3603705,BX03,172247,39405.79222,44.72,14.24,14.89,12.38,36.82,54.24,...,1437,5436,16.80,41.00,22.29,0.03,8.45,11.43,2.39,9.04


In [195]:
# How many community districts are in file? 
dist_pop = df_pop['cd_id'].value_counts().index

In [196]:
len (dist_pop)

59

In [197]:
# Manipulate data to get some information on demographics by Community District. 
# Think about who might be more likely to call 311

df_311_user = df_pop[['cd_id','Median household income (In 2014 Inflation Adjusted Dollars)',
                    '% Total Population: 25 to 34 Years', 
                    '% Total Population: 35 to 44 Years']]

# People who are between 25 to 44 Years

In [198]:
df_311_user.columns = ['Community District', 'Median 2014 Income', 'Age 25-34', 'Age 35-44']
df_311_user.head()

,Community District,Median 2014 Income,Age 25-34,Age 35-44
0,BX08,54224,15.04,11.32
1,BX12,44906,14.29,12.57
2,BX10,54962,13.74,12.78
3,BX11,47910,17.43,14.09
4,BX03,22041,14.89,12.38


In [199]:
# Infrastructure by Community District
os.system("curl -O http://cosmo.nyu.edu/~fb55/PUI2016/data/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv" )
os.system("mv " + "ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv " + os.getenv("PUIDATA"))
df_infras = pd.read_csv(PUIdata + "/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv")

In [200]:
# Check variables in file
df_infras.head()

,FIPS,Geographic Identifier,Qualifying Name,Households,Households: With An Internet Subscription,Households: Dial-Up Alone,Households: Dsl,Households: With Mobile Broadband,Households: Without Mobile Broadband,Households: Cable Modem,...,Households: Mobile Broadband Alone or With Dialup,Households: Internet Access Without A Subscription,Households: No Internet Access,% Households: With An Internet Subscription,Households.1,Households: Has A Computer,Households: With Dial-Up Internet Subscription Alone,Households: With A Broadband Internet Subscription,Households: Without An Internet Subscription,Households: No Computer
0,3603701,79500US3603701,"NYC-Bronx Community District 8--Riverdale, New...",42035,31795,444,2813,946,1867,19178,...,2168,2119,8121,75.64,42035,35048,404,30943,3701,6987
1,3603702,79500US3603702,"NYC-Bronx Community District 12--Wakefield, Ne...",44830,32243,178,849,405,444,18653,...,928,1891,10696,71.92,44830,36700,178,31435,5087,8130
2,3603703,79500US3603703,"NYC-Bronx Community District 10--Co-op City, N...",47050,32729,158,1863,398,1465,20044,...,639,2882,11439,69.56,47050,38700,158,32333,6209,8350
3,3603704,79500US3603704,NYC-Bronx Community District 11--Pelham Parkwa...,44922,32003,141,1478,474,1004,17917,...,1001,2722,10197,71.24,44922,37237,122,31278,5837,7685
4,3603705,79500US3603705,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,35503,88,1297,651,646,15847,...,1385,3312,18741,61.68,57556,42576,88,33408,9080,14980


In [201]:
# How many community districts are in file? 
len(df_infras['Qualifying Name'].unique())

55

In [202]:
# Manipulate data to get some information on internet/broadband useage by Community District
# Aggregate the mobile subscription data

In [203]:
internet_usage_by_commdist = df_infras[['Qualifying Name', 'Households: With Dial-Up Internet Subscription Alone', 
                              'Households: With A Broadband Internet Subscription']]
internet_usage_by_commdist.columns = ['Community District', 'Dialup Internet', 'Broadband Internet']
internet_usage_by_commdist['Has Internet'] = internet_usage_by_commdist['Dialup Internet'] + internet_usage_by_commdist['Broadband Internet']
internet_usage_by_commdist.drop(['Dialup Internet', 'Broadband Internet'], axis=1, inplace=True)
internet_usage_by_commdist.head(10)

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Community District,Has Internet
0,"NYC-Bronx Community District 8--Riverdale, New...",31347
1,"NYC-Bronx Community District 12--Wakefield, Ne...",31613
2,"NYC-Bronx Community District 10--Co-op City, N...",32491
3,NYC-Bronx Community District 11--Pelham Parkwa...,31400
4,"NYC-Bronx Community District 3 & 6--Belmont, N...",33496
5,"NYC-Bronx Community District 3 & 6--Belmont, N...",33496
6,"NYC-Bronx Community District 7--Bedford Park, ...",31021
7,NYC-Bronx Community District 5--Morris Heights...,25826
8,"NYC-Bronx Community District 4--Concourse, New...",28287
9,"NYC-Bronx Community District 9--Castle Hill, N...",45441


### 5. Inform your results by linking datasets

Now you want to link the three data frames to produce summary statistics for Community Districts which show a high number of complaints vs. Community Districts which show a lower number of complaints. Please keep in mind that the identifiers used for the linkage (Community Dostrict IDs) should be recored the same way. Use regular expressions to harmonize the identifiers if possible. The identifiers should look like BK01, BK02, etc. https://docs.python.org/2/library/re.html

In [204]:
# Harmonize identifier of dataframe 1

In [216]:
df1 = complaints

In [217]:
df1.columns

Index([u'Number of Complaints'], dtype='object')

In [ ]:
# cd_id = {'BRONX': 'BX', 'BROOKLYN': 'BK', 'MANHATTAN': 'MN', 'QUEENS': 'QN', 'STATEN ISLAND': 'SI'}


# complaints['cd_num'] = complaints['Community Board'].str[:2]
# complaints['cd_bor'] = complaints['Community Board'].str[3:]


# complaints.replace({'cd_bor': cd_id}, inplace=True)
# complaints['cd_id'] = complaints.cd_bor + complaints.cd_num
# complaints_update = complaints[["cd_id", "Number of Complaints"]]

# complaints_update.head()

In [220]:
# harmonize dataframe 2

In [222]:
df_311_user['Community District'].unique()

array(['BX08', 'BX12', 'BX10', 'BX11', 'BX03', 'BX06', 'BX07', 'BX05',
       'BX04', 'BX09', 'BX01', 'BX02', 'MN12', 'MN09', 'MN10', 'MN11111',
       'MN08', 'MN07', 'MN05', 'MN04', 'MN06', 'MN03', 'MN01', 'MN02',
       'SI03', 'SI02', 'SI01', 'BK01', 'BK04', 'BK03', 'BK02', 'BK06',
       'BK08', 'BK16', 'BK05', 'BK18', 'BK17', 'BK09', 'BK07', 'BK10',
       'BK12', 'BK14', 'BK15', 'BK11', 'BK13', 'QN01', 'QN03', 'QN07',
       'QN11', 'QN13', 'QN08', 'QN04', 'QN06', 'QN02', 'QN05', 'QN09',
       'QN12', 'QN10', 'QN14'], dtype=object)

In [223]:
df_311_user.replace('MN11111', 'MN11', inplace=True)
df_311_user['Community District'].unique()

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


array(['BX08', 'BX12', 'BX10', 'BX11', 'BX03', 'BX06', 'BX07', 'BX05',
       'BX04', 'BX09', 'BX01', 'BX02', 'MN12', 'MN09', 'MN10', 'MN11',
       'MN08', 'MN07', 'MN05', 'MN04', 'MN06', 'MN03', 'MN01', 'MN02',
       'SI03', 'SI02', 'SI01', 'BK01', 'BK04', 'BK03', 'BK02', 'BK06',
       'BK08', 'BK16', 'BK05', 'BK18', 'BK17', 'BK09', 'BK07', 'BK10',
       'BK12', 'BK14', 'BK15', 'BK11', 'BK13', 'QN01', 'QN03', 'QN07',
       'QN11', 'QN13', 'QN08', 'QN04', 'QN06', 'QN02', 'QN05', 'QN09',
       'QN12', 'QN10', 'QN14'], dtype=object)

In [224]:
# Harmonize dataframe 3